# Libraries, data acquisition and onehot encoding 

In [ ]:
import snowflake.connector
import logging
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier as sgbt
import xgboost as xgb
import optuna
from statistics import mean
import joblib
from os import environ
from PyQt5.QtCore.QUrl import query
from dotenv import load_dotenv

## Conection to snowflake 

In [3]:
load_dotenv()
for logger_name in ['snowflake.connector', 'botocore']:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    logger.addHandler(handler)

conn = snowflake.connector.connect(
    user = environ['USER'],
    password = environ['PASSWORD'],
    account = environ['ACCOUNT'],
    warehouse = environ['WAREHOUSE'],
    database = environ['DATABASE'],
    schema = environ['SCHEMA']
)

Snowflake Connector for Python Version: 3.12.2, Python Version: 3.10.14, Platform: Windows-10-10.0.22631-SP0
connect
__config
Connecting to GLOBAL Snowflake domain
This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
use_numpy: False
initialized
REST API object was created: ******.snowflakecomputing.com:443
authenticate
assertion content: *********
account=******, user=******, database=******, schema=******, warehouse=******, role=None, request_id=f1dcf2db-52f7-46bd-8d8c-19d8c3e78439
body['data']: {'CLIENT_APP_ID': 'PythonConnector', 'CLIENT_APP_VERSION': '3.12.2', 'SVN_REVISION': None, 'ACCOUNT_NAME': '******', 'LOGIN_NAME': '******', 'CLIENT_ENVIRONMENT': {'APPLICATION': 'PythonConnector', 'OS': 'Windows', 'OS_VERSION': 'Windows-10-10.0.22631-SP0', 'PYTHON_VERSION': '3.10.14', 'PYTHON_RUNTIME': 'CPyth

OperationalError: 250001: 250001: Could not connect to Snowflake backend after 2 attempt(s).Aborting

In [3]:
query = ('''
select 
    distinct(bp.id_match),
    bp.match_type_detail_name,
    bp.class_match,
    bp.total_players,
    ds.value_buyin as buyin_price,
    ds.value_rebuy as rebuy_price,
    ds.value_addon as addon_price,
    case 
        when extract(dow from bp.date_start_time_match_utc_local) = 0 then 'sunday'
        when extract(dow from bp.date_start_time_match_utc_local) = 1 then 'monday'
        when extract(dow from bp.date_start_time_match_utc_local) = 2 then 'tuesday'
        when extract(dow from bp.date_start_time_match_utc_local) = 3 then 'wednesday'
        when extract(dow from bp.date_start_time_match_utc_local) = 4 then 'thursday'
        when extract(dow from bp.date_start_time_match_utc_local) = 5 then 'friday'
        when extract(dow from bp.date_start_time_match_utc_local) = 6 then 'saturday'
    end as day_of_week,
    case
        when extract(hour from bp.date_start_time_match_utc_local) between 6 and 11 then 'morning'
        when extract(hour from bp.date_start_time_match_utc_local) between 12 and 17 then 'afternoon'
        when extract(hour from bp.date_start_time_match_utc_local) between 18 and 23 then 'evening'
        else 'dawn'
    end as period,
    cast(bp.gtd_match as integer) as gtd_match,
    round(bp.buyin_value, 2) as buyin_earned,
    round(bp.rebuy_value, 2) as rebuy_earned,
    round(bp.addon_value, 2) as addon_earned,
    bp.overlay_match,
    case
        when bp.gtd_match != 0 then ABS((bp.overlay_match / bp.gtd_match) * 100)
        else 0
    end as overlay_perc,
    case
        when bp.overlay_match < 0 then 0
        when bp.overlay_match = 0 then 1
        else bp.overlay_match
    end as overlay_match_bi,
    bp.duration_match
from 
    bi_product bp 
join
    (select distinct id_match, value_buyin, value_rebuy, value_addon
    from dim_union_summary) ds
    on bp.id_match = ds.id_match
where 
    bp.match_type_detail_name LIKE 'MTT%'
    AND buyin_value > 0
    AND grand_union = 113
    AND name_match NOT IN ('HG', 'LIVE', 'PVD', 'PVT', 'CLASS')
    AND duration_match > '00:05:00'
''')

In [4]:
data = pd.read_sql_query(query, conn)

C:\Users\maiap\AppData\Local\Temp\ipykernel_26492\3998575330.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)
cursor
executing SQL/command
query: [select distinct(bp.id_match), bp.match_type_detail_name, bp.class_match, bp.tota...]
binding: [select distinct(bp.id_match), bp.match_type_detail_name, bp.class_match, bp.tota...] with input=[None], processed=[{}]
sequence counter: 1
Request id: 944989b2-1e3b-42ec-8812-8d3732f900c7
running query [select distinct(bp.id_match), bp.match_type_detail_name, bp.class_match, bp.tota...]
is_file_transfer: True
_cmd_query
serialize_to_dict() called
sql=[select distinct(bp.id_match), bp.match_type_detail_name, bp.class_match, bp.tota...], sequence_id=[1], is_file_transfer=[False]
Opentelemtry otel injection failed because of: No module named 'opentelemetry'

In [5]:
data

,ID_MATCH,MATCH_TYPE_DETAIL_NAME,CLASS_MATCH,TOTAL_PLAYERS,BUYIN_PRICE,REBUY_PRICE,ADDON_PRICE,DAY_OF_WEEK,PERIOD,GTD_MATCH,BUYIN_EARNED,REBUY_EARNED,ADDON_EARNED,OVERLAY_MATCH,OVERLAY_PERC,OVERLAY_MATCH_BI,DURATION_MATCH
0,24069336,MTT-NLH,Micro,1648,3.0,3.0,3.0,monday,evening,10000,4944.0,2316.0,3933.0,-150.16,1.5016,0.0,08:07:03
1,22224395,MTT-NLH,Micro,929,3.0,3.0,3.0,wednesday,afternoon,4000,2787.0,1092.0,2325.0,0.00,0.0000,1.0,07:53:16
2,27575146,MTT-NLH,Micro,1227,0.6,0.6,0.6,tuesday,evening,1500,736.2,636.6,492.6,0.00,0.0000,1.0,06:19:16
3,26315514,MTT-NLH,Micro,761,3.0,3.0,3.0,wednesday,afternoon,4000,2283.0,702.0,1962.0,0.00,0.0000,1.0,08:02:16
4,21250424,MTT-NLH,Micro,58,1.0,1.0,0.0,tuesday,evening,50,58.0,29.0,0.0,0.00,0.0000,1.0,02:31:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27937,20893732,MTT-NLH,Medium,2,30.0,40.0,0.0,thursday,evening,3000,60.0,0.0,0.0,-2949.00,98.3000,0.0,02:10:53
27938,22930694,MTT-NLH,Micro,28,1.2,1.2,1.2,sunday,evening,66,33.6,28.8,24.0,0.00,0.0000,1.0,01:55:27
27939,24774985,MTT-PLO4,Micro,75,3.0,3.0,0.0,sunday,morning,200,225.0,105.0,0.0,0.00,0.0000,1.0,04:52:15
27940,23382998,MTT-PLO5,Micro,20,3.0,3.0,3.0,wednesday,afternoon,100,60.0,36.0,36.0,0.00,0.0000,1.0,02:03:56


In [10]:
encoder = OneHotEncoder(sparse_output=False)
data_transform = data.drop(['ID_MATCH', 'TOTAL_PLAYERS', 'BUYIN_EARNED',	'REBUY_EARNED',	'ADDON_EARNED', 'OVERLAY_MATCH','OVERLAY_PERC',	'DURATION_MATCH' ], axis=1)
cat_columns = ['MATCH_TYPE_DETAIL_NAME', 'CLASS_MATCH','DAY_OF_WEEK', 'PERIOD']
encoded_columns = encoder.fit_transform(data_transform[cat_columns])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(cat_columns))
data_final = pd.concat([data_transform.drop(cat_columns, axis=1), encoded_df], axis=1)

In [ ]:
data_final

In [ ]:
joblib.dump(encoder, '/content/drive/MyDrive/Colab Notebooks/new_encoder.pkl')

In [13]:
X = data_final.drop(columns=['OVERLAY_MATCH_BI'], axis=1)
y = data_final['OVERLAY_MATCH_BI']
X.columns

Index(['BUYIN_PRICE', 'REBUY_PRICE', 'ADDON_PRICE', 'GTD_MATCH',
       'MATCH_TYPE_DETAIL_NAME_MTT-NLH', 'MATCH_TYPE_DETAIL_NAME_MTT-PLO4',
       'MATCH_TYPE_DETAIL_NAME_MTT-PLO5', 'MATCH_TYPE_DETAIL_NAME_MTT-PLO6',
       'CLASS_MATCH_High', 'CLASS_MATCH_Low', 'CLASS_MATCH_Medium',
       'CLASS_MATCH_Micro', 'DAY_OF_WEEK_friday', 'DAY_OF_WEEK_monday',
       'DAY_OF_WEEK_saturday', 'DAY_OF_WEEK_sunday', 'DAY_OF_WEEK_thursday',
       'DAY_OF_WEEK_tuesday', 'DAY_OF_WEEK_wednesday', 'PERIOD_afternoon',
       'PERIOD_dawn', 'PERIOD_evening', 'PERIOD_morning'],
      dtype='object')

In [14]:
encoder.get_feature_names_out()

array(['MATCH_TYPE_DETAIL_NAME_MTT-NLH',
       'MATCH_TYPE_DETAIL_NAME_MTT-PLO4',
       'MATCH_TYPE_DETAIL_NAME_MTT-PLO5',
       'MATCH_TYPE_DETAIL_NAME_MTT-PLO6', 'CLASS_MATCH_High',
       'CLASS_MATCH_Low', 'CLASS_MATCH_Medium', 'CLASS_MATCH_Micro',
       'DAY_OF_WEEK_friday', 'DAY_OF_WEEK_monday', 'DAY_OF_WEEK_saturday',
       'DAY_OF_WEEK_sunday', 'DAY_OF_WEEK_thursday',
       'DAY_OF_WEEK_tuesday', 'DAY_OF_WEEK_wednesday', 'PERIOD_afternoon',
       'PERIOD_dawn', 'PERIOD_evening', 'PERIOD_morning'], dtype=object)

In [15]:
def cross_validation_optuna(trial, X, y, n_splits):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.8)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)

    model = xgb.XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        device='cuda',  # Utilizar a GPU com CUDA
        random_state=42
    )

    kfold = KFold(n_splits=n_splits)
    f1_scores_split_lst = []
    all_pred = []
    all_reals = []

    for idx, (idx_train, idx_test) in enumerate(kfold.split(X)):
        X_split_train = X.iloc[idx_train, :]
        y_split_train = y.iloc[idx_train]

        sm = SMOTE(random_state=42)
        X_split_train, y_split_train = sm.fit_resample(X_split_train, y_split_train)

        model.fit(X_split_train, y_split_train.values.flatten())

        X_split_test = X.iloc[idx_test, :]
        y_split_test = y.iloc[idx_test]

        predt_validation = model.predict(X_split_test)

        f1_split = f1_score(y_split_test, predt_validation, pos_label=0)
        f1_scores_split_lst.append(f1_split)

        all_pred.extend(predt_validation)
        all_reals.extend(y_split_test.values.flatten())

        print(f'F1-Score da classe 0 no split {idx}: {f1_split}')

    print('\nRelatório de Classificação Final:\n', classification_report(all_reals, all_pred))

    importance = model.feature_importances_
    importances = pd.Series(importance, index=X_split_train.columns)

    return mean(f1_scores_split_lst), model, importances

def optimize_hyperparameters(X, y):
    study = optuna.create_study(direction='maximize')

    def objective(trial):
        f1_mean, model, importances = cross_validation_optuna(trial, X, y, n_splits=10)
        trial.set_user_attr("model", model)
        trial.set_user_attr("importances", importances)
        return f1_mean

    study.optimize(objective, n_trials=40)

    # Show the best parameters
    print("Best hyperparameters: ", study.best_params)
    print("Best score: ", study.best_value)

    # Recuperar o melhor modelo e as importâncias das features
    best_trial = study.best_trial
    best_model = best_trial.user_attrs["model"]
    best_importances = best_trial.user_attrs["importances"]

    return study.best_params

In [16]:
best_param = optimize_hyperparameters(X, y)

[I 2024-10-02 14:40:47,340] A new study created in memory with name: no-name-b3b354fa-6685-46c6-805e-c6362932898f
C:\Users\maiap\anaconda3\envs\suprema-env\lib\site-packages\xgboost\core.py:158: UserWarning: [14:40:48] WARNING: D:\bld\xgboost-split_1727635012356\work\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


F1-Score da classe 0 no split 0: 0.36155606407322655
F1-Score da classe 0 no split 1: 0.3689776733254994
F1-Score da classe 0 no split 2: 0.3741794310722101
F1-Score da classe 0 no split 3: 0.3590859630032644
F1-Score da classe 0 no split 4: 0.3818615751789976
F1-Score da classe 0 no split 5: 0.39211136890951276
F1-Score da classe 0 no split 6: 0.3673469387755102
F1-Score da classe 0 no split 7: 0.40518256772673733
F1-Score da classe 0 no split 8: 0.38199780461031835


[I 2024-10-02 14:40:51,836] Trial 0 finished with value: 0.37711229160870413 and parameters: {'n_estimators': 53, 'max_depth': 5, 'learning_rate': 0.21246769245075017, 'subsample': 0.8234694404001808, 'colsample_bytree': 0.8270505143303661}. Best is trial 0 with value: 0.37711229160870413.


F1-Score da classe 0 no split 9: 0.3788235294117647

Relatório de Classificação Final:
               precision    recall  f1-score   support

         0.0       0.31      0.49      0.38      3330
         1.0       0.93      0.85      0.89     24612

    accuracy                           0.81     27942
   macro avg       0.62      0.67      0.63     27942
weighted avg       0.85      0.81      0.82     27942

F1-Score da classe 0 no split 0: 0.4112808460634548
F1-Score da classe 0 no split 1: 0.3862559241706161
F1-Score da classe 0 no split 2: 0.40471092077087795
F1-Score da classe 0 no split 3: 0.37753882915173237
F1-Score da classe 0 no split 4: 0.37142857142857144
F1-Score da classe 0 no split 5: 0.3880926130099228
F1-Score da classe 0 no split 6: 0.36900369003690037
F1-Score da classe 0 no split 7: 0.4394829612220917
F1-Score da classe 0 no split 8: 0.4022222222222222


[I 2024-10-02 14:41:08,460] Trial 1 finished with value: 0.3950971232014337 and parameters: {'n_estimators': 178, 'max_depth': 12, 'learning_rate': 0.575143085172302, 'subsample': 0.6280778386716578, 'colsample_bytree': 0.5004117235318419}. Best is trial 1 with value: 0.3950971232014337.


...

F1-Score da classe 0 no split 9: 0.4029126213592233

Relatório de Classificação Final:
               precision    recall  f1-score   support

         0.0       0.32      0.52      0.40      3330
         1.0       0.93      0.85      0.89     24612

    accuracy                           0.81     27942
   macro avg       0.63      0.69      0.64     27942
weighted avg       0.86      0.81      0.83     27942

F1-Score da classe 0 no split 0: 0.42080378250591016
F1-Score da classe 0 no split 1: 0.3933253873659118
F1-Score da classe 0 no split 2: 0.4105378704720088
F1-Score da classe 0 no split 3: 0.38016528925619836
F1-Score da classe 0 no split 4: 0.37320574162679426
F1-Score da classe 0 no split 5: 0.38505096262740657
F1-Score da classe 0 no split 6: 0.39263803680981596
F1-Score da classe 0 no split 7: 0.42502951593860683
F1-Score da classe 0 no split 8: 0.4079822616407982


[I 2024-10-02 14:50:09,130] Trial 38 finished with value: 0.3995924476985966 and parameters: {'n_estimators': 134, 'max_depth': 17, 'learning_rate': 0.13334853448692238, 'subsample': 0.897017791959533, 'colsample_bytree': 0.6407438935438339}. Best is trial 32 with value: 0.40526004522193604.


F1-Score da classe 0 no split 9: 0.40718562874251496

Relatório de Classificação Final:
               precision    recall  f1-score   support

         0.0       0.33      0.51      0.40      3330
         1.0       0.93      0.86      0.89     24612

    accuracy                           0.82     27942
   macro avg       0.63      0.69      0.65     27942
weighted avg       0.86      0.82      0.83     27942

F1-Score da classe 0 no split 0: 0.41784037558685444
F1-Score da classe 0 no split 1: 0.3908872901678657
F1-Score da classe 0 no split 2: 0.40869565217391307
F1-Score da classe 0 no split 3: 0.38497652582159625
F1-Score da classe 0 no split 4: 0.36585365853658536
F1-Score da classe 0 no split 5: 0.40418118466898956
F1-Score da classe 0 no split 6: 0.39856801909307876
F1-Score da classe 0 no split 7: 0.4408352668213457
F1-Score da classe 0 no split 8: 0.412348401323043


[I 2024-10-02 14:50:22,533] Trial 39 finished with value: 0.4034687567510694 and parameters: {'n_estimators': 90, 'max_depth': 15, 'learning_rate': 0.047930711119924106, 'subsample': 0.7688385904734505, 'colsample_bytree': 0.7172387453149325}. Best is trial 32 with value: 0.40526004522193604.


F1-Score da classe 0 no split 9: 0.4105011933174224

Relatório de Classificação Final:
               precision    recall  f1-score   support

         0.0       0.33      0.52      0.40      3330
         1.0       0.93      0.86      0.89     24612

    accuracy                           0.82     27942
   macro avg       0.63      0.69      0.65     27942
weighted avg       0.86      0.82      0.83     27942

Best hyperparameters:  {'n_estimators': 121, 'max_depth': 12, 'learning_rate': 0.05060213376478348, 'subsample': 0.8323861385223982, 'colsample_bytree': 0.6845221707634495}
Best score:  0.40526004522193604
